In [61]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import re
%matplotlib inline   

## Change elements below

In [62]:
expts=['071018_Ii']
wells=[['D03','D04','D05','D06','D07','D08']]
sites=[9]

framemax=28      #35 for Nov2018 on, 29 for 071018

inpath='Put_input_path_here'

outfile='Output_path/___means.csv'


# Get mean Fluor intensity for each cell

In [63]:
def getGFPmean(e,w,s,cell):
    
    #parse name to get path to input file:
    cellnum=re.search(r"Cell_([0-9]*)", cell)
    cellnum=cellnum.group(1)
    sitePath=inpath+'/well'+w+'/Adapt_v1.189/'+e+'_'+w+'_w2_'+'s'+str(s+1)+'_Output/'
    fname=sitePath+str(cellnum)+'_Output/MeanFluorescenceIntensity.csv' 
    
    #get mean of first frame fluor intensity
    filedf=pd.read_csv(fname, index_col=0)
    firstframe=filedf.iloc[:,0]
    gfpmean=firstframe.mean()    #dictionary of cellname (key), cell mean (value) pairs  
    
    return gfpmean
    
    

## Get stats dataframe for each cell and remove cells not present in all frames

In [64]:
##Get cell names and indices in file

def wellstats(expt,well,sitenum):
    sitelist=[]
    for s in range(sitenum):
        
        inputfilename=inpath+'/well'+well+'/Adapt_v1.189/'+expt+'_'+well+'_w2_'+'s'+str(s+1)+'_Output/morphology.txt'
               
        if os.path.getsize(inputfilename) > 0:     #to deal with empty morph files.
            all_data = pd.read_csv(inputfilename, sep='\t')
            all_data=all_data.rename(columns={' ':'Cell'})
            cellindices=[]
            celllabels=[]
            for i in range(len(all_data.index)):
                if 'Cell_' in all_data.Cell.iloc[i]:
                    cellindices.append(i)
                    celllabels.append(all_data.Cell.iloc[i])

            ##Get frame rows for each cell and make dataframe for each
            ##Name each dataframe well_site_cell and put into dictionary

            last=len(all_data.index)
            cellindices=cellindices+[last]
            base=cellindices[len(cellindices)-2]
            cellindices[:0]=[0]

            celldict={}

            for n, c in enumerate(cellindices):
                if n==0:
                    dfslice=all_data.iloc[(base+cellindices[n+1]-cellindices[n]+1):(base+cellindices[n+2]-cellindices[n+1]),:]
                    name=expt+'_'+well+'_s'+str(s+1)+'_'+celllabels[n]
                    celldict[name]=dfslice
                elif n<len(cellindices)-2:
                    dfslice=all_data.iloc[(base+cellindices[n+1]-cellindices[n]):(base+cellindices[n+2]-cellindices[n+1]),:]
                    name=expt+'_'+well+'_s'+str(s+1)+'_'+celllabels[n]
                    celldict[name]=dfslice


            ##Go through cell dictionary and remove elements with less than framemax number of frames

            fullcells={}

            for name, df in celldict.iteritems():
                #print name+' '+str(len(df))
                
                
                if len(df)>=framemax:    #for 071018
                    fullcells[name]=df
                
                
                #if len(df)==framemax:     #for Nov-Dec 2018
                #    fullcells[name]=df

            #Get means of each stat for each cell 

            dflist=[]
            for name, df in fullcells.iteritems():
                juststats=df.iloc[:,1:]
                statsdf=pd.DataFrame(juststats.mean(), columns=[name])
                statsdf=statsdf.T
                ##get GFP mean fluorescence and append as column here:
                gfpmean=getGFPmean(expt,well,s,name)
                statsdf['FluorMean']=gfpmean
                #append full means df for each cell:
                dflist.append(statsdf)
                
            sitedf=pd.concat(dflist)
            sitelist.append(sitedf)
                
    welldf=pd.concat(sitelist)    
    return welldf
        

## Main

In [65]:
##Process morphology.txt files and get stat means for each cell

totallist=[]
for i, e in enumerate(expts):
    exptlist=[]
    for w in wells[i]:
        welldf=wellstats(e,w,sites[i])
        exptlist.append(welldf)
        
    exptdf=pd.concat(exptlist)
    totallist.append(exptdf)    
    
statmeansdf=pd.concat(totallist)  #dataframe with all cells as rows, and all 7 means as columns

##output full csv
statmeansdf.to_csv(outfile)
        
        